In [1]:
import random 
import pandas as pd
import hopsworks

In [2]:
BACKFILL = False # if True then we intend to Backfill with a .csv file

### Generating Synthetic data:

In [3]:
def generate_bank_note(variance_max, variance_min, skewness_max, skewness_min,
                      curtosis_max, curtosis_min, entropy_max, entropy_min, class_):
    
    """
    Returns a single bank note with specs within the ranges specified
    """
    df = pd.DataFrame({
        'variance' : [random.uniform(variance_max, variance_min)],
        'skewness' : [random.uniform(skewness_max, skewness_min)],
        'curtosis' : [random.uniform(curtosis_max, curtosis_min)],
        'entropy' : [random.uniform(entropy_max, entropy_min)],
        'output' : [class_]
    })
    
    
    return df

def get_random_bank_note():
    
    """
    Returns a DataFrame containing one random bank note
    """
    class_1 = generate_bank_note(6,-7,12,-13,17,-5,2,-8,1)
    class_0 = generate_bank_note(5.5,-5.5,10,-10.5,15.5,-4.5,2.3,-7.35,0)
    
    random_num = random.uniform(0,2)
    if random_num >= 1:
        df = class_1
    else:
        df = class_0
        
    return df

In [4]:
if BACKFILL == True:
    df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt',header=None,
                names = ['variance', 'skewness', 'curtosis', 'entropy', 'output'])
else:
    df = get_random_bank_note()

In [5]:
df

,variance,skewness,curtosis,entropy,output
0,1.121676,-7.225372,12.23489,-1.145395,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1 non-null      float64
 1   skewness  1 non-null      float64
 2   curtosis  1 non-null      float64
 3   entropy   1 non-null      float64
 4   output    1 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 168.0 bytes


In [7]:
df.columns

Index(['variance', 'skewness', 'curtosis', 'entropy', 'output'], dtype='object')

### Hopsworks Auth using API key:

In [8]:
proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/39062
Connected. Call `.close()` to terminate connection gracefully.


### Create and Write to a feature group - PK

In [9]:
bank_note_df = fs.get_or_create_feature_group(name = 'bank_notes_dataframe',
                                             version = 1,
                                             primary_key = ['variance', 'skewness', 'curtosis', 'entropy'],
                                             description = 'Authenticating Bank Note dataset', )
feature_cols = ['variance', 'skewness', 'curtosis', 'entropy', 'output']
bank_note_df.insert(df[feature_cols])

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/39062/jobs/named/bank_notes_dataframe_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2381943c3a0>, None)